In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
df1 = pd.read_csv("nau_art1.csv", delimiter = "|", header = None)
df2 = pd.read_csv("nau_art2.csv", delimiter = "|", header = None)
df3 = pd.read_csv("nau_art3.csv", delimiter = "|", header = None)
df4 = pd.read_csv("nau_art4.csv", delimiter = "|", header = None)

frames = [df1, df2, df3, df4]

df_main = pd.concat(frames, ignore_index= True)

In [ ]:
#remove columns without title, text or url
df_main = df_main.dropna(subset=[0])
df_main = df_main.dropna(subset=[5])
df_main = df_main.dropna(subset=[6])

In [ ]:
#set date and remove all entries from 2021
df_main[2] = df_main[2].str.replace("Januar", "01.")
df_main[2] = df_main[2].str.replace("Februar", "02.")
df_main[2] = df_main[2].str.replace("März", "03.")
df_main[2] = df_main[2].str.replace("April", "04.")
df_main[2] = df_main[2].str.replace("Mai", "05.")
df_main[2] = df_main[2].str.replace("Juni", "06.")
df_main[2] = df_main[2].str.replace("Juli", "07.")
df_main[2] = df_main[2].str.replace("August", "08.")
df_main[2] = df_main[2].str.replace("September", "09.")
df_main[2] = df_main[2].str.replace("Oktober", "10.")
df_main[2] = df_main[2].str.replace("November", "11.")
df_main[2] = df_main[2].str.replace("Dezember", "12.")
df_main[2] = df_main[2].str.replace(" ", "")
df_main[2] = pd.to_datetime(df_main[2], format="%d.%m.%Y")
df_main[7] = df_main[2].dt.year
df_main = df_main[df_main[7] != 2021]
df_main = df_main.drop([7], axis =1)

In [ ]:
#drop duplicates
df_main = df_main.drop_duplicates(keep = "last")

In [ ]:
#extracting categories
categories = df_main[6].str.split("/",expand=True)
categories[6] = categories[3].str.len()
categories[7] = categories[6].astype(int)
categories[8] = np.where(categories[7] < 20, categories[3],"")

df_main[7] = categories[8]
df_main[8] = categories[4]

In [ ]:
df_main[9] = df_main[7]
df_main[10] = df_main[8]

In [ ]:
#resorts creation
with open("ressorts_cat.bin", "rb") as data:
    ressorts = pickle.load(data)

df_main[9].replace(ressorts, inplace=True)
df_main[10].replace(ressorts, inplace=True)

In [ ]:
#delete lines with non-speaking category indication
with open("to_delete.bin", "rb")as data:
    to_delete = pickle.load(data)
    
df_main = df_main[~df_main[8].isin(to_delete)]
df_main = df_main[~df_main[7].isin(to_delete)]

In [ ]:
#avoid duplicated in category creation
df_main[11] = np.where(df_main[9] == df_main[10], "", df_main[10])
df_main = df_main.drop(columns = 10)

In [ ]:
#reset index
df_main.reset_index(drop=True)

In [ ]:
df_main[12] = "nau"
df_main[4] = df_main[4].str.replace("http\S+", "",regex = True)
df_main[4] = df_main[4].str.replace("Bild", "")

In [ ]:
df_main.to_csv("nau_artikel.csv", header = None, sep = "|", index = False)